In [1]:
import subprocess
import platform
import lmdb
import Image
import sys
import copy 
import numpy as np
import PIL.Image
import matplotlib.pyplot as plt
from StringIO import StringIO
from caffe.proto import caffe_pb2
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn import decomposition
from sklearn import preprocessing
from sklearn.neighbors.kde import KernelDensity
from scipy.stats import multivariate_normal
from scipy.stats.mstats import rankdata
from sklearn.externals import joblib
from __future__ import division
%matplotlib inline
import caffe
caffe.set_mode_gpu()
caffe.set_device(0)

print "OS:     ", platform.platform()
print "Python: ", sys.version.split("\n")[0]
print "CUDA:   ", subprocess.Popen(["nvcc","--version"], stdout=subprocess.PIPE).communicate()[0].split("\n")[3]
print "LMDB:   ", ".".join([str(i) for i in lmdb.version()])

OS:      Linux-3.13.0-65-generic-x86_64-with-Ubuntu-14.04-trusty
Python:  2.7.6 (default, Jun 22 2015, 17:58:13) 
CUDA:    Cuda compilation tools, release 7.5, V7.5.17
LMDB:    0.9.18


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
kind = 'siamese'
mid_layer = 'conv5_3'
next_layer = 'pool5'
addr = mid_layer+'/'+kind+'/'
print addr
num_pca = 4

conv5_3/siamese/


In [3]:
def load_all_middle_features(net, middle_layer_name, num_batch, istest):
    # Loading all features from middle layer
    # num_batch is not the batch_size. (batch_size is set before in training)
    features = []
    labels = []
    acc = 0
    middle_layer_shape = net.blobs[middle_layer_name].data.shape
    for i in range(0, num_batch):
        out = net.forward(blobs=[middle_layer_name,'label'])
        features.append(copy.copy(np.reshape(out[middle_layer_name],(middle_layer_shape[0],-1) )))
        labels.append(copy.copy(out['label']))
        if istest==True :
            acc = acc + net.blobs['accuracy_test_top01'].data #accuracy_test_top01
    acc = acc / num_batch
    if istest==True:
        print('Accuracy of test is {:f}'.format(acc))
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    print('Features are loaded')
    print('Percentage of men is {:f}'.format(np.sum(labels)/labels.shape[0]))
    return features, labels

def my_pca(features, labels, num_pca):
    mean_feature = np.mean(features, axis=0)
    normalized_features = features - mean_feature
    #pca = decomposition.PCA(svd_solver='randomized')
    pca = decomposition.IncrementalPCA(batch_size=50)
    pca.n_components = num_pca
    pca.fit(normalized_features)
    print('Remained variance is:') 
    print(pca.explained_variance_ratio_) 
    X_reduced = pca.transform(normalized_features)
    print('PCA is fitted')
    if num_pca==2:
        plt.figure()
        for c, i in zip("rgbcmyk", [0, 1, 2, 3, 4, 5, 6]):
            plt.scatter(X_reduced[labels == i, 0], X_reduced[labels == i, 1], c=c)
        plt.title('PCA of features')
    return pca, X_reduced, mean_feature

def pirvacy_ek(X_reduced, X_reduced_noisy, noise_var):
    # Compute Privacy Measure
    # kde = KernelDensity(kernel='gaussian', bandwidth=1).fit(X_reduced_noisy)
    numdata = X_reduced.shape[0]
    d = np.array([X_reduced,]*numdata) - np.transpose(np.array([X_reduced_noisy,]*numdata),(1,0,2)) #order?
    e = multivariate_normal.pdf(d,np.zeros([X_reduced.shape[1],]), noise_var*np.eye(X_reduced.shape[1]))
    e = e.T #added now
    r = e/np.max(e,0)
    m = np.sum(r>.8,0)
    ranks = 1 + numdata - rankdata(r,0)
    trueranks = np.diag(ranks)
    res = np.maximum(m,trueranks)
    #res = m
    print min(res)
    print max(res)
    print np.mean(res)
    print np.std(res)
    print sum(res==1)
    print sum(m>trueranks)
    print numdata
    res = res / numdata
    priv = np.mean(res)
    var = np.var(res)
    print('Expected ek-privacy is {:f}'.format(priv))
    return priv, var

def compute_acc(net, middle_layer_name, next_layer_name, reconst_all_middles, reconst_all_labels, num_batch):
    newacc = 0
    for i in range(0, num_batch):
        net.blobs[middle_layer_name].data[...] = reconst_all_middles[i,:,:,:,:]
        net.blobs['label'].data[...] = reconst_all_labels[i,:]
        net.forward(start=next_layer_name, end='accuracy_test_top01') #accuracy_test_top01 for gender
        newacc = newacc + net.blobs['accuracy_test_top01'].data
    newacc = newacc / num_batch
    print('Accuracy of test on reconstructed features from PCA is {:f}'.format(newacc))
    return newacc

def reduce_dim_test(pca, features, labels, mean_feature):
    normalized_features = features - mean_feature
    X_reduced = pca.transform(normalized_features)
    print('extracted PCA from train data is fitted to test data')
    if pca.n_components==2:
        plt.figure()
        for c, i in zip("rgbcmyk", [0, 1, 2, 3, 4, 5, 6]):
            plt.scatter(X_reduced[labels == i, 0], X_reduced[labels == i, 1], c=c)
        plt.title('PCA of features')
    return X_reduced

def load_data_into_lmdb(lmdb_name, features, labels=None):
    env = lmdb.open(lmdb_name, map_size=features.nbytes*2)
    
    features = features[:,:,None,None]
    for i in range(features.shape[0]):
        datum = caffe.proto.caffe_pb2.Datum()
        
        datum.channels = features.shape[1]
        datum.height = 1
        datum.width = 1
        
        if features.dtype == np.int:
            datum.data = features[i].tostring()
        if features.dtype == np.float: 
            datum.float_data.extend(features[i].flat)
        else:
            raise Exception("features.dtype unknown.")
        
        if labels is not None:
            datum.label = int(labels[i])
        
        str_id = '{:08}'.format(i)
        with env.begin(write=True) as txn:
            txn.put(str_id, datum.SerializeToString())

In [4]:
# Set your network here. You should see the caffemodels folder and pick the right caffemodel and set it. 
# Also lmdb of train.prototxt should be set to lmdb of FR. Please make a copy of it and dont ruine itself.
# Surely you will have the memory problem. Good Luck
net = caffe.Net(addr+'train-f.prototxt', addr+'caffemodels/_iter_3000.caffemodel', caffe.TRAIN)

In [14]:
512*14*14

100352

In [5]:
features, labels = load_all_middle_features(net, mid_layer, 500, False) # 500 is number of batches

Features are loaded
Percentage of men is 50.089800


In [6]:
# load pc
pca = joblib.load(addr+'pca/pca_'+str(num_pca)+'.pkl')
mean_feature = np.load(addr+'pca/mean_feature_'+str(num_pca)+'.npy')

In [59]:
a = np.ones([100352,])
b = a - mean_feature

In [60]:
c = pca.transform(b)
c

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 25.55669227,  -0.11051862,  -1.44156744,  -1.31595093]])

In [62]:
d = pca.inverse_transform(c)
d

array([[  7.54095407e-04,   3.92926398e-04,  -1.42494414e-05, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [69]:
e = d + mean_feature
e

array([[  8.60998055e-04,   4.10755983e-04,  -8.79854709e-07, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [43]:
np.dot(np.ones([100352,]),np.transpose(pca.components_))

array([ 26.50697765,  -0.08840955,  -1.43441972,  -1.3689867 ])

In [57]:
np.dot(a,pca.components_)

array([[  7.79629849e-04,   4.06460204e-04,  -1.49570668e-05, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [27]:
np.ones([3,])

array([ 1.,  1.,  1.])

In [7]:
X_reduced = reduce_dim_test(pca, features, labels, mean_feature)

extracted PCA from train data is fitted to test data


In [8]:
X_reduced.shape

(5000, 4)

In [ ]:
# add noise to PCA
noise = np.random.multivariate_normal(np.zeros([num_pca,]), noise_var*np.eye(num_pca), features.shape[0])
X_reduced_noisy = X_reduced + noise

In [ ]:
# Reconstruct PCA
temp_reconst = pca.inverse_transform(X_reduced_noisy) 
noisy_reconst = temp_reconst + mean_feature      

In [ ]:
# reshaping to the correct form
middle_layer_shape = net.blobs[middle_layer_name].data.shape
reconst_all_middles = np.reshape(noisy_reconst, (num_batch,)+middle_layer_shape )
reconst_all_labels = np.reshape(labels, (num_batch,)+(middle_layer_shape[0],) )
print('Reconstruction is done from PCA projections')

In [ ]:
load_data_into_lmdb(mid_layer+"_lmdb", reconst_all_middles.astype(float), reconst_all_labels) 